In [ ]:
!python --version

Python 3.10.12


In [ ]:
!pip install pandasql

In [ ]:
!pip install sqlalchemy==1.4.16

# Importando dados e bibliotecas
---

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pandasql as ps
from pandasql import sqldf

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
caminho_3 = ''

In [ ]:
df_3 = pd.read_csv(caminho_3)

In [ ]:
df_3.head()

,Unnamed: 0,Player,Age,G,GS,rAtt,rYds,rTD,r1D,rLng,rY/A,rY/g,Fmb,Year
0,0,Stephen Davis,27,16,16,356,1432,5,75,32,4.0,89.5,6,2001
1,1,Corey Dillon,27,16,16,340,1315,10,69,96,3.9,82.2,5,2001
2,2,LaDainian Tomlinson,22,16,16,339,1236,10,68,54,3.6,77.3,8,2001
3,3,Curtis Martin,28,16,16,333,1513,10,78,47,4.5,94.6,2,2001
4,4,Priest Holmes,28,16,16,327,1555,8,81,41,4.8,97.2,4,2001


# Removendo as colunas `rYds`, `rLng`, `Unnamed: 0`, `rY/A` e `rY/g`
---

In [ ]:
df_3.drop(['rYds', 'rLng', 'Unnamed: 0', 'rY/A', 'rY/g'], axis=1)

,Player,Age,G,GS,rAtt,rTD,r1D,Fmb,Year
0,Stephen Davis,27,16,16,356,5,75,6,2001
1,Corey Dillon,27,16,16,340,10,69,5,2001
2,LaDainian Tomlinson,22,16,16,339,10,68,8,2001
3,Curtis Martin,28,16,16,333,10,78,2,2001
4,Priest Holmes,28,16,16,327,8,81,4,2001
...,...,...,...,...,...,...,...,...,...
7511,Jonathan Williams,29,1,0,1,0,0,0,2023
7512,Mike Williams,29,3,3,1,0,0,0,2023
7513,Robert Woods,31,14,11,1,0,0,0,2023
7514,Logan Woodside,28,1,0,1,0,0,0,2023


# Montando uma query que mostre o jogador (`Player`) com maior proporção de touchdowns correndo (`Rushing touchdowns`) em relação a quantidade de tentativas de corrida (`rAtt`), levando em conta apenas o ano de 2001
---

In [ ]:
query = '''
SELECT Player, (rTD * 1.0 / rAtt) AS Proporcao_TD_por_Att
FROM df_3
WHERE Year = 2001
ORDER BY Proporcao_TD_por_Att DESC
'''

result = sqldf(query, locals())
print(result)

                 Player  Proporcao_TD_por_Att
0             Tim Seder              1.000000
1            Tim Dwight              0.500000
2         Donald Driver              0.333333
3            Paul Smith              0.250000
4          Leon Johnson              0.200000
..                  ...                   ...
299         Tony Taylor              0.000000
300  Marques Tuiasosopo              0.000000
301       Justin Watson              0.000000
302       Alvis Whitted              0.000000
303       Frank Wycheck              0.000000

[304 rows x 2 columns]


# Consultando o jogador (`Player`) com maior quantidade de tentativas de corrida (`rAtt`) em cada ano (`Year`), levando em conta apenas o período entre 2002 e 2004 e, em caso de empate, ordenando por ordem alfabética
---

In [ ]:
query = '''
SELECT Year, Player, MAX(rAtt) AS Max_rAtt
FROM (
    SELECT Year, Player, rAtt,
           ROW_NUMBER() OVER (PARTITION BY Year ORDER BY rAtt DESC, Player) AS rn
    FROM df_3
    WHERE Year BETWEEN 2002 AND 2004
)
WHERE rn = 1
GROUP BY Year
'''

result = sqldf(query, locals())
print(result)

   Year          Player  Max_rAtt
0  2002  Ricky Williams       383
1  2003  Ricky Williams       392
2  2004   Curtis Martin       371


# Consultando o jogador (`Player`) com maior quantidade de fumbles (`Fmb`) no ano (`Year`) de 2002 e, em caso de empate, ordenando por ordem alfabética
---

In [ ]:
query = '''
SELECT Year, Player, MAX(Fmb) AS Max_Fmb
FROM (
    SELECT Year, Player, Fmb,
           ROW_NUMBER() OVER (PARTITION BY Year ORDER BY Fmb DESC, Player) AS rn
    FROM df_3
    WHERE Year = 2002
)
GROUP BY Year
'''

result = sqldf(query, locals())
print(result)

   Year            Player  Max_Fmb
0  2002  Daunte Culpepper       23


# Consultando o jogador (`Player`) com menor taxa de fumbles (`Fmb`) considerando o período entre 2001 e 2004 e, em caso de empate, ordenando por ordem alfabética
---

In [ ]:
query = '''
SELECT Year, Player, MIN(Fmb) AS Min_Fmb
FROM (
    SELECT Year, Player, Fmb,
           ROW_NUMBER() OVER (PARTITION BY Year ORDER BY Fmb ASC, Player) AS rn
    FROM df_3
    WHERE Year BETWEEN 2001 AND 2004
)
WHERE rn = 1
GROUP BY Year
'''

result = sqldf(query, locals())
print(result)

   Year           Player  Min_Fmb
0  2001    Aaron Stecker        0
1  2002  Adrian Peterson        0
2  2003   Adrian Murrell        0
3  2004  Adrian Peterson        0


# Consultando o ano (`Year`) em que os jogadores (`Player`) possuiam maior média de idade e, em caso de empate, ordenando do mais recente para o mais antigo
---

In [ ]:
query = '''
SELECT Year, AVG(Age) AS Avg_Age
FROM df_3
GROUP BY Year
ORDER BY Avg_Age DESC;
'''

result = sqldf(query, locals())
print(result)

    Year    Avg_Age
0   2005  27.227564
1   2007  27.192547
2   2002  27.151420
3   2004  27.069069
4   2006  27.047771
5   2003  27.033537
6   2001  26.973684
7   2008  26.922330
8   2009  26.743590
9   2010  26.690625
10  2015  26.512821
11  2011  26.417957
12  2016  26.369697
13  2017  26.250000
14  2021  26.245283
15  2013  26.244300
16  2018  26.225806
17  2023  26.208092
18  2014  26.190323
19  2019  26.185629
20  2022  26.158470
21  2012  26.069401
22  2020  26.010753
